# Deep Agents: Building Complex Agents for Long-Horizon Tasks

In this notebook, we'll explore **Deep Agents** - a new approach to building AI agents that can handle complex, multi-step tasks over extended periods. We'll implement all four key elements of Deep Agents while building on our Personal Wellness Assistant use case.

**Learning Objectives:**
- Understand the four key elements of Deep Agents: Planning, Context Management, Subagent Spawning, and Long-term Memory
- Implement each element progressively using the `deepagents` package
- Learn to use Skills for progressive capability disclosure
- Use the `deepagents-cli` for interactive agent sessions

## Table of Contents:

- **Breakout Room #1:** Deep Agent Foundations
  - Task 1: Dependencies & Setup
  - Task 2: Understanding Deep Agents
  - Task 3: Planning with Todo Lists
  - Task 4: Context Management with File Systems
  - Task 5: Basic Deep Agent
  - Question #1 & Question #2
  - Activity #1: Build a Research Agent

- **Breakout Room #2:** Advanced Features & Integration
  - Task 6: Subagent Spawning
  - Task 7: Long-term Memory Integration
  - Task 8: Skills - On-Demand Capabilities
  - Task 9: Using deepagents-cli
  - Task 10: Building a Complete Deep Agent System
  - Question #3 & Question #4
  - Activity #2: Build a Wellness Coach Agent

---
# 🤝 Breakout Room #1
## Deep Agent Foundations

## Task 1: Dependencies & Setup

Before we begin, make sure you have:

1. **API Keys** for:
   - Anthropic (default for Deep Agents) or OpenAI
   - LangSmith (optional, for tracing)
   - Tavily (optional, for web search)

2. **Dependencies installed** via `uv sync`

3. **For the CLI** (Task 9): `uv pip install deepagents-cli`

### Environment Setup

You can either:
- Create a `.env` file with your API keys (recommended):
  ```
  ANTHROPIC_API_KEY=your_key_here
  OPENAI_API_KEY=your_key_here
  LANGCHAIN_API_KEY=your_key_here
  ```
- Or enter them interactively when prompted

In [1]:
# Core imports
import os
import getpass
from uuid import uuid4
from typing import Annotated, TypedDict, Literal

import nest_asyncio
nest_asyncio.apply()  # Required for async operations in Jupyter

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

def get_api_key(env_var: str, prompt: str) -> str:
    """Get API key from environment or prompt user."""
    value = os.environ.get(env_var, "")
    if not value:
        value = getpass.getpass(prompt)
        if value:
            os.environ[env_var] = value
    return value

In [12]:
# Set Anthropic API Key (default for Deep Agents)
anthropic_key = get_api_key("ANTHROPIC_API_KEY", "Anthropic API Key: ")
if anthropic_key:
    print("Anthropic API key set")
else:
    print("Warning: No Anthropic API key configured")

Anthropic API key set


In [13]:
# Optional: OpenAI for alternative models and subagents
openai_key = get_api_key("OPENAI_API_KEY", "OpenAI API Key (press Enter to skip): ")
if openai_key:
    print("OpenAI API key set")
else:
    print("OpenAI API key not configured (optional)")

OpenAI API key set


In [14]:
# Optional: LangSmith for tracing
langsmith_key = get_api_key("LANGCHAIN_API_KEY", "LangSmith API Key (press Enter to skip): ")

if langsmith_key:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = f"AIE9 - Deep Agents - {uuid4().hex[0:8]}"
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")
else:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled")

LangSmith tracing enabled. Project: AIE9 - Deep Agents - 0e1882f7


In [15]:
# Verify deepagents installation
from deepagents import create_deep_agent
print("deepagents package imported successfully!")

# Test with a simple agent
test_agent = create_deep_agent()
result = test_agent.invoke({
    "messages": [{"role": "user", "content": "Say 'Deep Agents ready!' in exactly those words."}]
})
print(result["messages"][-1].content)

/Users/martynakosciukiewicz/Documents/source/AIE9/07_Deep_Agents/.venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


deepagents package imported successfully!
Deep Agents ready!


## Task 2: Understanding Deep Agents

**Deep Agents** represent a shift from simple tool-calling loops to sophisticated agents that can handle complex, long-horizon tasks. They address four key challenges:

### The Four Key Elements

| Element | Challenge Addressed | Implementation |
|---------|---------------------|----------------|
| **Planning** | "What should I do?" | Todo lists that persist task state |
| **Context Management** | "What do I know?" | File systems for storing/retrieving info |
| **Subagent Spawning** | "Who can help?" | Task tool for delegating to specialists |
| **Long-term Memory** | "What did I learn?" | LangGraph Store for cross-session memory |

### Deep Agents vs Traditional Agents

```
Traditional Agent Loop:
┌─────────────────────────────────────┐
│  User Query                         │
│       ↓                             │
│  Think → Act → Observe → Repeat     │
│       ↓                             │
│  Response                           │
└─────────────────────────────────────┘
Problems: Context bloat, no delegation,
          loses track of complex tasks

Deep Agent Architecture:
┌─────────────────────────────────────────────────────────┐
│                    Deep Agent                           │
├─────────────────────────────────────────────────────────┤
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐   │
│  │   PLANNING   │  │   CONTEXT    │  │   MEMORY     │   │
│  │              │  │  MANAGEMENT  │  │              │   │
│  │ write_todos  │  │              │  │   Store      │   │
│  │ update_todo  │  │  read_file   │  │  namespace   │   │
│  │ list_todos   │  │  write_file  │  │  get/put     │   │
│  │              │  │  edit_file   │  │              │   │
│  └──────────────┘  │  ls          │  └──────────────┘   │
│                    └──────────────┘                     │
│  ┌──────────────────────────────────────────────────┐   │
│  │              SUBAGENT SPAWNING                   │   │
│  │                                                  │   │
│  │  task(prompt, tools, model, system_prompt)       │   │
│  │       ↓              ↓              ↓            │   │
│  │  ┌────────┐    ┌────────┐    ┌────────┐          │   │
│  │  │Research│    │Writing │    │Analysis│          │   │
│  │  │Subagent│    │Subagent│    │Subagent│          │   │
│  │  └────────┘    └────────┘    └────────┘          │   │
│  └──────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────┘
```

### When to Use Deep Agents

| Use Case | Traditional Agent | Deep Agent |
|----------|-------------------|------------|
| Simple Q&A | ✅ | Overkill |
| Single-step tool use | ✅ | Overkill |
| Multi-step research | ⚠️ May lose track | ✅ |
| Complex projects | ❌ Context overflow | ✅ |
| Parallel task execution | ❌ | ✅ |
| Long-running sessions | ❌ | ✅ |

### Key Insight: "Planning is Context Engineering"

Deep Agents treat planning not as a separate phase, but as **context engineering**:
- Todo lists aren't just task trackers—they're **persistent context** about what to do
- File systems aren't just storage—they're **extended memory** beyond the context window
- Subagents aren't just helpers—they're **context isolation** to prevent bloat

## Task 3: Planning with Todo Lists

The first key element of Deep Agents is **Planning**. Instead of trying to hold all task state in the conversation, Deep Agents use structured todo lists.

### Why Todo Lists?

1. **Persistence**: Tasks survive across conversation turns
2. **Visibility**: Both agent and user can see progress
3. **Structure**: Clear tracking of what's done vs pending
4. **Recovery**: Agent can resume from where it left off

### Todo List Tools

| Tool | Purpose |
|------|----------|
| `write_todos` | Create a structured task list |
| `update_todo` | Mark tasks as complete/in-progress |
| `list_todos` | View current task state |

In [16]:
from langchain_core.tools import tool
from typing import List, Optional
import json

# Simple in-memory todo storage for demonstration (dict)
# In production, Deep Agents use persistent storage
TODO_STORE = {}

@tool
def write_todos(todos: List[dict]) -> str:
    """Create a list of todos for tracking task progress.
    
    Args:
        todos: List of todo items, each with 'title' and optional 'description'
    
    Returns:
        Confirmation message with todo IDs
    """
    created = []
    for i, todo in enumerate(todos):
        todo_id = f"todo_{len(TODO_STORE) + i + 1}"
        TODO_STORE[todo_id] = {
            "id": todo_id,
            "title": todo.get("title", "Untitled"),
            "description": todo.get("description", ""),
            "status": "pending"
        }
        created.append(todo_id)
    return f"Created {len(created)} todos: {', '.join(created)}"

@tool
def update_todo(todo_id: str, status: Literal["pending", "in_progress", "completed"]) -> str:
    """Update the status of a todo item.
    
    Args:
        todo_id: The ID of the todo to update
        status: New status (pending, in_progress, completed)
    
    Returns:
        Confirmation message
    """
    if todo_id not in TODO_STORE:
        return f"Todo {todo_id} not found"
    TODO_STORE[todo_id]["status"] = status
    return f"Updated {todo_id} to {status}"

@tool
def list_todos() -> str:
    """List all todos with their current status.
    
    Returns:
        Formatted list of all todos
    """
    if not TODO_STORE:
        return "No todos found"
    
    result = []
    for todo_id, todo in TODO_STORE.items():
        status_emoji = {"pending": "⬜", "in_progress": "🔄", "completed": "✅"}
        emoji = status_emoji.get(todo["status"], "❓")
        result.append(f"{emoji} [{todo_id}] {todo['title']} ({todo['status']})")
    return "\n".join(result)

print("Todo tools defined!")

Todo tools defined!


In [8]:
# Test the todo tools
TODO_STORE.clear()  # Reset for demo

# Create some wellness todos
result = write_todos.invoke({
    "todos": [
        {"title": "Assess current sleep patterns", "description": "Review user's sleep schedule and quality"},
        {"title": "Research sleep improvement strategies", "description": "Find evidence-based techniques"},
        {"title": "Create personalized sleep plan", "description": "Combine findings into actionable steps"},
    ]
})
print(result)
print("\nCurrent todos:")
print(list_todos.invoke({}))

Created 3 todos: todo_1, todo_3, todo_5

Current todos:
⬜ [todo_1] Assess current sleep patterns (pending)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


In [9]:
# Simulate progress
update_todo.invoke({"todo_id": "todo_1", "status": "completed"})
update_todo.invoke({"todo_id": "todo_2", "status": "in_progress"})

print("After updates:")
print(list_todos.invoke({}))

After updates:
✅ [todo_1] Assess current sleep patterns (completed)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


## Task 4: Context Management with File Systems

The second key element is **Context Management**. Deep Agents use file systems to:

1. **Offload large content** - Store research, documents, and results to disk
2. **Persist across sessions** - Files survive beyond conversation context
3. **Share between subagents** - Subagents can read/write shared files
4. **Prevent context overflow** - Large tool results automatically saved to disk

### Automatic Context Management

Deep Agents automatically handle context limits:
- **Large result offloading**: Tool results >20k tokens → saved to disk
- **Proactive offloading**: At 85% context capacity → agent saves state to disk
- **Summarization**: Long conversations get summarized while preserving intent

### File System Tools

| Tool | Purpose |
|------|----------|
| `ls` | List directory contents |
| `read_file` | Read file contents |
| `write_file` | Create/overwrite files |
| `edit_file` | Make targeted edits |

In [17]:
import os
from pathlib import Path

# Create a workspace directory for our agent
WORKSPACE = Path("workspace")
WORKSPACE.mkdir(exist_ok=True)

@tool
def ls(path: str = ".") -> str:
    """List contents of a directory.
    
    Args:
        path: Directory path to list (default: current directory)
    
    Returns:
        List of files and directories
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"Directory not found: {path}"
    
    items = []
    for item in sorted(target.iterdir()):
        prefix = "[DIR]" if item.is_dir() else "[FILE]"
        size = f" ({item.stat().st_size} bytes)" if item.is_file() else ""
        items.append(f"{prefix} {item.name}{size}")
    
    return "\n".join(items) if items else "(empty directory)"

@tool
def read_file(path: str) -> str:
    """Read contents of a file.
    
    Args:
        path: Path to the file to read
    
    Returns:
        File contents
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    return target.read_text()

@tool
def write_file(path: str, content: str) -> str:
    """Write content to a file (creates or overwrites).
    
    Args:
        path: Path to the file to write
        content: Content to write to the file
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    target.parent.mkdir(parents=True, exist_ok=True)
    target.write_text(content)
    return f"Wrote {len(content)} characters to {path}"

@tool
def edit_file(path: str, old_text: str, new_text: str) -> str:
    """Edit a file by replacing text.
    
    Args:
        path: Path to the file to edit
        old_text: Text to find and replace
        new_text: Replacement text
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    
    content = target.read_text()
    if old_text not in content:
        return f"Text not found in {path}"
    
    new_content = content.replace(old_text, new_text, 1)
    target.write_text(new_content)
    return f"Updated {path}"

print("File system tools defined!")
print(f"Workspace: {WORKSPACE.absolute()}")

File system tools defined!
Workspace: /Users/martynakosciukiewicz/Documents/source/AIE9/07_Deep_Agents/workspace


In [8]:
# Test the file system tools
print("Current workspace contents:")
print(ls.invoke({"path": "."}))

Current workspace contents:
[FILE] personalized_sleep_improvement_plan.md (8527 bytes)
[DIR] research
[FILE] sleep_improvement_action_plan.md (9712 bytes)
[FILE] sleep_research_analysis.md (22890 bytes)
[FILE] stress_management_guide.md (8549 bytes)


In [8]:
# Create a research notes file
notes = """# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations
"""

result = write_file.invoke({"path": "research/sleep_notes.md", "content": notes})
print(result)

# Verify it was created
print("\nResearch directory:")
print(ls.invoke({"path": "research"}))

Wrote 242 characters to research/sleep_notes.md

Research directory:
[FILE] sleep_notes.md (242 bytes)


In [13]:
# Read and edit the file
print("File contents:")
print(read_file.invoke({"path": "research/sleep_notes.md"}))

File contents:
# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations



## Task 5: Basic Deep Agent

Now let's create a basic Deep Agent using the `deepagents` package. This combines:
- Planning (todo lists)
- Context management (file system)
- A capable LLM backbone

### Configuring the FilesystemBackend

Deep Agents come with **built-in file tools** (`ls`, `read_file`, `write_file`, `edit_file`). To control where files are stored, we configure a `FilesystemBackend`:

```python
from deepagents.backends import FilesystemBackend

backend = FilesystemBackend(
    root_dir="/path/to/workspace",
    virtual_mode=True  # REQUIRED to actually sandbox files!
)
```

**Critical: `virtual_mode=True`**
- Without `virtual_mode=True`, agents can still write anywhere on the filesystem!
- The `root_dir` alone does NOT restrict file access
- `virtual_mode=True` blocks paths with `..`, `~`, and absolute paths outside root

In [18]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Configure the filesystem backend to use our workspace directory
# IMPORTANT: virtual_mode=True is required to actually restrict paths to root_dir
# Without it, agents can still write anywhere on the filesystem!
workspace_path = Path("workspace").absolute()
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True  # This is required to sandbox file operations!
)

# Combine our custom tools (for todo tracking)
# Note: Deep Agents has built-in file tools (ls, read_file, write_file, edit_file)
# that will use the configured FilesystemBackend
custom_tools = [
    write_todos,
    update_todo,
    list_todos,
]

# Create a basic Deep Agent
wellness_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=custom_tools,
    backend=filesystem_backend,  # Configure where files are stored
    system_prompt="""You are a Personal Wellness Assistant that helps users improve their health.

When given a complex task:
1. First, create a todo list to track your progress
2. Work through each task, updating status as you go
3. Save important findings to files for reference
4. Provide a clear summary when complete

Be thorough but concise. Always explain your reasoning."""
)

print(f"Basic Deep Agent created!")
print(f"File operations sandboxed to: {workspace_path}")

Basic Deep Agent created!
File operations sandboxed to: /Users/martynakosciukiewicz/Documents/source/AIE9/07_Deep_Agents/workspace


In [15]:
# Reset todo store for fresh demo
TODO_STORE.clear()

# Test with a multi-step wellness task
result = wellness_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please create a personalized sleep improvement plan for me and save it to a file."""
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Perfect! I've created your comprehensive, personalized sleep improvement plan and saved it to `/personalized_sleep_improvement_plan.md`. 

## 🎉 Your Sleep Transformation Plan is Ready!

Your plan specifically addresses your three main challenges:

### **🎯 For Your Inconsistent Bedtime (10pm-1am):**
- Starts with establishing a consistent wake time (more important than bedtime initially)
- Gradually narrows your bedtime window from ±3 hours to ±30 minutes
- Uses your natural sleepiness cues to find your optimal bedtime

### **📱 For Your Bedtime Phone Use:**
- Graduated approach: 30 → 60 → 90 minute phone-free periods
- Specific replacement activities (physical books, journaling, gentle stretching)
- Creates phone-free charging station outside bedroom

### **😴 For Your Morning Tiredness:**
- Strategic morning light exposure (within 15 minutes of waking)
- Consistent wake time to anchor circadian rhythm
- Temperature and environment optimization for deeper sleep

## **🚀 St

In [18]:
# Check what the agent created
print("Todo list after task:")
print(list_todos.invoke({}))

print("\n" + "="*50)
print("\nWorkspace contents:")
# List files in the workspace directory
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    else:
        print(f"  [DIR] {f.name}/")

Todo list after task:
✅ [todo_1] Analyze current sleep issues (completed)
✅ [todo_3] Research evidence-based sleep hygiene practices (completed)
✅ [todo_5] Create personalized sleep schedule recommendations (completed)
✅ [todo_7] Develop screen time management strategies (completed)
✅ [todo_9] Design morning routine improvements (completed)
✅ [todo_11] Compile comprehensive sleep improvement plan (completed)
✅ [todo_13] Save plan to file (completed)
✅ [todo_8] Research sleep schedule optimization strategies (completed)
✅ [todo_10] Research screen time management for sleep (completed)
✅ [todo_12] Research solutions for morning fatigue (completed)
✅ [todo_14] Research additional evidence-based sleep hygiene practices (completed)
✅ [todo_16] Develop implementation timeline and practical tips (completed)
✅ [todo_18] Compile comprehensive research summary (completed)


Workspace contents:
  [FILE] .gitkeep (0 bytes)
  [FILE] personalized_sleep_improvement_plan.md (5292 bytes)
  [DIR] resear

---
## ❓ Question #1:

What are the **trade-offs** of using todo lists for planning? Consider:
- When might explicit planning overhead slow things down?
- How granular should todo items be?
- What happens if the agent creates todos but never completes them?

##### Answer:
I can see how creating a to-do list could be a waste of resources in the case of simple tasks not requiring many steps to complete. In such cases, creating that additional structure with to-dos generates more costs without much  benefit. It's kind of like using LLM-generated responses to queries where you could use a supervised classifier if you have enough data to train it well.

I think the to-do item granularity should be based on the scope of the user query and such that every item can be an action in itself, e.g. if in the example of sleep issue we have a task "analyze current sleep issues" there's no need to create subtasks for it (unless the user query was specifically about doing a deep analysis of sleep issues and nothing more). 

If an agent created to-dos and never completed them it probably means that those steps were not required to complete the task, or the agent made a mistake in the planning or executing process. Yo could probably analyse the execution steps to diagnose the issue.

## ❓ Question #2:

How would you design a **context management strategy** for a wellness agent that:
- Needs to reference a large health document (16KB)
- Tracks user metrics over time
- Must remember user conditions (allergies, medications) for safety

What goes in files vs. in the prompt? What should never be offloaded?

##### Answer:
Although the  health document is not massive, I might chunk it and put it in a DB for RAG. In this way the document is stored externally and all its content is not loaded into the context window every time but only chunks relevant to the user query.

For the metrics, I would put them into a long-term memory store in such a way that I could query it keying on metric type, user_id and other interesting info for performance analysis. If relevant to the task, user info could be pulled and injected into the prompt.

As for things like the user conditions, I would store these in the long-term memory, since it's user specific info. I would also build a safety guardrail into the prompt underlying that the agent must consider the user's health conditions when generating response. This should be never offloaded as its central to the system's safety.

---
## 🏗️ Activity #1: Build a Research Agent

Build a Deep Agent that can research a wellness topic and produce a structured report.

### Requirements:
1. Create todos for the research process
2. Read from the HealthWellnessGuide.txt in the data folder
3. Save findings to a structured markdown file
4. Update todo status as tasks complete

### Test prompt:
"Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."

In [19]:
# Define a function that will be used as a tool to read the wellness guide
@tool
def read_wellness_guide() -> str:
    """Read the Wellness Guide file.
    
    Args:
        path: Path to the Wellness Guide file 
    """
    path = Path("data/HealthWellnessGuide.txt")
    if path.exists():   
        return path.read_text()
    else:
        return f"File not found: {path}"

In [20]:
# define agent tools
research_tools = [
    write_todos,
    update_todo,
    list_todos,
    read_wellness_guide,
]

# Create the agent with a research-focused system prompt
wellness_research_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=research_tools,
    backend=filesystem_backend,  
    system_prompt="""You are a Personal Wellness Research Assistant that helps users improve their health by creating a structured report on a requested topic.

When given a complex task:
1. First, create a todo list to track your progress in the research process
2. Work through each task, updating status as you go
3. Update todo status as tasks complete

Requirements:
    - Read from the HealthWellnessGuide.txt in the data folder and use it as a reference for the research
    - Save findings to a structured markdown file
    - Provide a clear summary when you complete the task

Be thorough but concise. Always explain your reasoning. Ground your responses in the HealthWellnessGuide.txt."""
)

In [21]:
# Test with the stress management research task
TODO_STORE.clear()

research_test = wellness_research_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."""
    }]
})

print(research_test["messages"][-1].content)

KeyboardInterrupt: 

In [13]:
# Check what the agent created
print(list_todos.invoke({}))

print("\n" + "="*50)
print("\nWorkspace contents:")
# List files in the workspace directory
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    else:
        print(f"  [DIR] {f.name}/")

✅ [todo_1] Read the Health Wellness Guide (completed)
✅ [todo_3] Research stress management techniques (completed)
✅ [todo_5] Compile at least 5 strategies (completed)
✅ [todo_7] Create comprehensive markdown guide (completed)
✅ [todo_9] Provide summary (completed)


Workspace contents:
  [FILE] personalized_sleep_improvement_plan.md (8527 bytes)
  [DIR] research/
  [FILE] sleep_improvement_action_plan.md (9712 bytes)
  [FILE] sleep_research_analysis.md (22890 bytes)
  [FILE] stress_management_guide.md (8549 bytes)


---
# 🤝 Breakout Room #2
## Advanced Features & Integration

## Task 6: Subagent Spawning

The third key element is **Subagent Spawning**. This allows a Deep Agent to delegate tasks to specialized subagents.

### Why Subagents?

1. **Context Isolation**: Each subagent has its own context window, preventing bloat
2. **Specialization**: Different subagents can have different tools/prompts
3. **Parallelism**: Multiple subagents can work simultaneously
4. **Cost Optimization**: Use cheaper models for simpler subtasks

### How Subagents Work

```
Main Agent
    ├── task("Research sleep science", model="gpt-4o-mini")
    │       └── Returns: Summary of findings
    │
    ├── task("Analyze user's sleep data", tools=[analyze_tool])
    │       └── Returns: Analysis results
    │
    └── task("Write recommendations", system_prompt="Be concise")
            └── Returns: Final recommendations
```

Key benefit: The main agent only receives **summaries**, not all the intermediate context!

In [22]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Define specialized subagent configurations
# Note: Subagents inherit the backend from the parent agent
research_subagent = {
    "name": "research-agent",
    "description": "Use this agent to research wellness topics in depth. It can read documents and synthesize information.",
    "system_prompt": """You are a wellness research specialist. Your job is to:
1. Find relevant information in provided documents
2. Synthesize findings into clear summaries
3. Cite sources when possible

Be thorough but concise. Focus on evidence-based information.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",  # Cheaper model for research
}

writing_subagent = {
    "name": "writing-agent",
    "description": "Use this agent to create well-structured documents, plans, and guides.",
    "system_prompt": """You are a wellness content writer. Your job is to:
1. Take research findings and turn them into clear, actionable content
2. Structure information for easy understanding
3. Use formatting (headers, bullets, etc.) effectively

Write in a supportive, encouraging tone.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "anthropic:claude-sonnet-4-20250514",
}

print("Subagent configurations defined!")

Subagent configurations defined!


In [23]:
# Create a coordinator agent that can spawn subagents
coordinator_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[write_todos, update_todo, list_todos],
    backend=filesystem_backend,  # Use the same backend - subagents inherit it
    subagents=[research_subagent, writing_subagent],
    system_prompt="""You are a Wellness Project Coordinator. Your role is to:
1. Break down complex wellness requests into subtasks
2. Delegate research to the research-agent
3. Delegate content creation to the writing-agent
4. Coordinate the overall workflow using todos

Use subagents for specialized work rather than doing everything yourself.
This keeps the work organized and the results high-quality."""
)

print("Coordinator agent created with subagent capabilities!")

Coordinator agent created with subagent capabilities!


In [21]:
# Reset for demo
TODO_STORE.clear()

# Test the coordinator with a complex task
result = coordinator_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Create a comprehensive morning routine guide for better energy.
        
The guide should:
1. Research the science behind morning routines
2. Include practical steps for exercise, nutrition, and mindset
3. Be saved as a well-formatted markdown file"""
    }]
})

print("Coordinator response:")
print(result["messages"][-1].content)

Coordinator response:
## 🌅 Your Comprehensive Morning Routine Guide is Complete!

I've successfully created a detailed, science-backed morning routine guide that combines cutting-edge research with practical, actionable strategies. Here's what you now have:

### **📋 Complete Guide Features:**

✅ **Scientific Foundation** - Research on circadian rhythms, exercise benefits, nutrition science, and mindset practices  
✅ **Practical Exercise Plans** - 3 levels (10-30 minutes) with specific exercises and progressions  
✅ **Detailed Nutrition Guidelines** - Meal structures, timing, hydration protocols, and supplementation  
✅ **Mindset Practices** - Meditation, gratitude, journaling, and visualization techniques  
✅ **Implementation Strategies** - Habit formation science and troubleshooting guides  
✅ **Lifestyle Customizations** - Tailored approaches for professionals, parents, students, remote workers, shift workers, and seniors  
✅ **Sample Templates** - Ready-to-use routines from 15 minut

In [22]:
# Check the results
print("Final todo status:")
print(list_todos.invoke({}))

print("\nGenerated files in workspace:")
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

Final todo status:
✅ [todo_1] Research the science behind morning routines (completed)
✅ [todo_3] Create comprehensive morning routine guide (completed)
✅ [todo_5] Save the guide as a markdown file (completed)

Generated files in workspace:
  [FILE] .gitkeep (0 bytes)
  [FILE] comprehensive_morning_routine_guide.md (25117 bytes)
  [FILE] morning_routine_guide.md (54068 bytes)
  [FILE] personalized_sleep_improvement_plan.md (5292 bytes)
  [DIR] research/
  [FILE] sleep_hygiene_research.md (15886 bytes)


## Task 7: Long-term Memory Integration

The fourth key element is **Long-term Memory**. Deep Agents integrate with LangGraph's Store for persistent memory across sessions.

### Memory Types in Deep Agents

| Type | Scope | Use Case |
|------|-------|----------|
| **Thread Memory** | Single conversation | Current session context |
| **User Memory** | Across threads, per user | User preferences, history |
| **Shared Memory** | Across all users | Common knowledge, learned patterns |

### Integration with LangGraph Store

Deep Agents can use the same `InMemoryStore` (or `PostgresStore`) we learned in Session 6:

In [24]:
from langgraph.store.memory import InMemoryStore

# Create a memory store
memory_store = InMemoryStore()

# Store user profile
user_id = "user_alex"
profile_namespace = (user_id, "profile")

memory_store.put(profile_namespace, "name", {"value": "Alex"})
memory_store.put(profile_namespace, "goals", {
    "primary": "improve energy levels",
    "secondary": "better sleep"
})
memory_store.put(profile_namespace, "conditions", {
    "dietary": ["vegetarian"],
    "medical": ["mild anxiety"]
})
memory_store.put(profile_namespace, "preferences", {
    "exercise_time": "morning",
    "communication_style": "detailed"
})

print(f"Stored profile for {user_id}")

# Retrieve and display
for item in memory_store.search(profile_namespace):
    print(f"  {item.key}: {item.value}")

Stored profile for user_alex
  name: {'value': 'Alex'}
  goals: {'primary': 'improve energy levels', 'secondary': 'better sleep'}
  conditions: {'dietary': ['vegetarian'], 'medical': ['mild anxiety']}
  preferences: {'exercise_time': 'morning', 'communication_style': 'detailed'}


In [25]:
# Create memory-aware tools
from langgraph.store.base import BaseStore

@tool
def get_user_profile(user_id: str) -> str:
    """Retrieve a user's wellness profile from long-term memory.
    
    Args:
        user_id: The user's unique identifier
    
    Returns:
        User profile as formatted text
    """
    namespace = (user_id, "profile")
    items = list(memory_store.search(namespace))
    
    if not items:
        return f"No profile found for {user_id}"
    
    result = [f"Profile for {user_id}:"]
    for item in items:
        result.append(f"  {item.key}: {item.value}")
    return "\n".join(result)

@tool
def save_user_preference(user_id: str, key: str, value: str) -> str:
    """Save a user preference to long-term memory.
    
    Args:
        user_id: The user's unique identifier
        key: The preference key
        value: The preference value
    
    Returns:
        Confirmation message
    """
    namespace = (user_id, "preferences")
    memory_store.put(namespace, key, {"value": value})
    return f"Saved preference '{key}' for {user_id}"

print("Memory tools defined!")

Memory tools defined!


In [26]:
# Create a memory-enhanced agent
memory_tools = [
    get_user_profile,
    save_user_preference,
    write_todos,
    update_todo,
    list_todos,
]

memory_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=memory_tools,
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a Personal Wellness Assistant with long-term memory.

At the start of each conversation:
1. Check the user's profile to understand their goals and conditions
2. Personalize all advice based on their profile
3. Save any new preferences they mention

Always reference stored information to show you remember the user."""
)

print("Memory-enhanced agent created!")

Memory-enhanced agent created!


In [26]:
# Test the memory agent
TODO_STORE.clear()

result = memory_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Hi! My user_id is user_alex. What exercise routine would you recommend for me?"
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Hi Alex! Great to see you again. Based on your profile, I remember that your main goals are to improve your energy levels and get better sleep, and you prefer morning workouts. I also see you manage mild anxiety and follow a vegetarian diet.

Here's a personalized exercise routine that aligns perfectly with your goals and preferences:

## Morning Energy-Boosting Routine (30-40 minutes)

### **Monday, Wednesday, Friday - Cardio + Light Strength**
- **5 min warm-up**: Dynamic stretching or light walking
- **20 min moderate cardio**: Brisk walking, jogging, or cycling (great for energy without overstimulation)
- **10 min bodyweight strength**: Push-ups, squats, planks, lunges
- **5 min cool-down**: Gentle stretching and deep breathing (helps with anxiety management)

### **Tuesday, Thursday - Yoga Flow**
- **30 min morning yoga sequence**: Focus on energizing poses like Sun Salutations, Warrior poses, and gentle backbends
- This combination promotes both energy and better 

## Task 8: Skills - On-Demand Capabilities

**Skills** are a powerful feature for progressive capability disclosure. Instead of loading all tools upfront, agents can load specialized capabilities on demand.

### Why Skills?

1. **Context Efficiency**: Don't waste context on unused tool descriptions
2. **Specialization**: Skills can include detailed instructions for specific tasks
3. **Modularity**: Easy to add/remove capabilities
4. **Discoverability**: Agent can browse available skills

### SKILL.md Format

Skills are defined in markdown files with YAML frontmatter:

```markdown
---
name: skill-name
description: What this skill does
version: 1.0.0
tools:
  - tool1
  - tool2
---

# Skill Instructions

Detailed steps for how to use this skill...
```

In [27]:
# Let's look at the skills we created
skills_dir = Path("skills")

print("Available skills:")
for skill_dir in skills_dir.iterdir():
    if skill_dir.is_dir():
        skill_file = skill_dir / "SKILL.md"
        if skill_file.exists():
            content = skill_file.read_text()
            # Extract name and description from frontmatter
            lines = content.split("\n")
            name = ""
            desc = ""
            for line in lines:
                if line.startswith("name:"):
                    name = line.split(":", 1)[1].strip()
                if line.startswith("description:"):
                    desc = line.split(":", 1)[1].strip()
            print(f"  - {name}: {desc}")

Available skills:
  - meal-planning: Create personalized meal plans based on dietary needs and preferences
  - wellness-assessment: Assess user wellness goals and create personalized recommendations


In [28]:
# Read the wellness-assessment skill
skill_content = Path("skills/wellness-assessment/SKILL.md").read_text()
print(skill_content)

---
name: wellness-assessment
description: Assess user wellness goals and create personalized recommendations
version: 1.0.0
tools:
  - read_file
  - write_file
---

# Wellness Assessment Skill

You are conducting a comprehensive wellness assessment. Follow these steps:

## Step 1: Gather Information
Ask the user about:
- Current health goals (weight, fitness, stress, sleep)
- Any medical conditions or limitations
- Current exercise routine (or lack thereof)
- Dietary preferences and restrictions
- Sleep patterns and quality
- Stress levels and sources

## Step 2: Analyze Responses
Review the user's answers and identify:
- Primary wellness priority
- Secondary goals
- Potential barriers to success
- Existing healthy habits to build on

## Step 3: Create Assessment Report
Write a wellness assessment report to `workspace/wellness_assessment.md` containing:
- Summary of current wellness state
- Identified strengths
- Areas for improvement
- Recommended focus areas (prioritized)
- Suggeste

In [28]:
# Create a skill-aware tool
@tool
def load_skill(skill_name: str) -> str:
    """Load a skill's instructions for a specialized task.
    
    Available skills:
    - wellness-assessment: Assess user wellness and create recommendations
    - meal-planning: Create personalized meal plans
    
    Args:
        skill_name: Name of the skill to load
    
    Returns:
        Skill instructions
    """
    skill_path = Path(f"skills/{skill_name}/SKILL.md")
    if not skill_path.exists():
        available = [d.name for d in Path("skills").iterdir() if d.is_dir()]
        return f"Skill '{skill_name}' not found. Available: {', '.join(available)}"
    
    return skill_path.read_text()

print("Skill loader defined!")

Skill loader defined!


In [29]:
# Create an agent that can load and use skills
skill_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        load_skill,
        write_todos,
        update_todo,
        list_todos,
    ],
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a wellness assistant with access to specialized skills.

When a user asks for something that matches a skill:
1. Load the appropriate skill using load_skill()
2. Follow the skill's instructions carefully
3. Save outputs as specified in the skill

Available skills:
- wellness-assessment: For comprehensive wellness evaluations
- meal-planning: For creating personalized meal plans

If no skill matches, use your general wellness knowledge."""
)

print("Skill-aware agent created!")

Skill-aware agent created!


In [31]:
# Test with a skill-appropriate request
TODO_STORE.clear()

result = skill_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "I'd like a wellness assessment. I'm a 35-year-old office worker who sits most of the day, has trouble sleeping, and wants to lose 15 pounds. I'm vegetarian and have no major health conditions."
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
## Your Wellness Assessment Summary

Based on your profile as a 35-year-old sedentary office worker looking to lose 15 pounds while dealing with sleep issues, I've created a comprehensive wellness assessment. Here are the key findings and recommendations:

### **Top Priority: Sleep Optimization** 
Your sleep issues are the foundation that needs addressing first - poor sleep disrupts weight loss hormones and energy levels needed for other changes.

### **Your Action Plan:**

**🟢 Start Today:**
1. **Sleep hygiene setup** - consistent bedtime, no screens 1 hour before bed
2. **Hourly movement breaks** - stand and move for 2-3 minutes every hour
3. **Hydration baseline** - water upon waking, keep bottle at desk

**🟡 Next 1-2 Weeks:**
1. **Daily 20-30 minute walks** plus 2-3 bodyweight strength sessions
2. **Consistent sleep schedule** (same bedtime/wake time daily)
3. **Food logging** for one week to identify patterns

**🔴 1-3 Month Goals:**
1. **150+ minutes cardio weekly*

## Task 9: Using deepagents-cli

The `deepagents-cli` provides an interactive terminal interface for working with Deep Agents.

### Installation

```bash
uv pip install deepagents-cli
# or
pip install deepagents-cli
```

### Key Features

| Feature | Description |
|---------|-------------|
| **Interactive Sessions** | Chat with your agent in the terminal |
| **Conversation Resume** | Pick up where you left off |
| **Human-in-the-Loop** | Approve or reject agent actions |
| **File System Access** | Agent can read/write to your filesystem |
| **Remote Sandboxing** | Run in isolated Docker containers |

### Basic Usage

```bash
# Start an interactive session
deepagents

# Resume a previous conversation
deepagents --resume

# Use a specific model
deepagents --model openai:gpt-4o

# Enable human-in-the-loop approval
deepagents --approval-mode full
```

### Example Session

```
$ deepagents

Welcome to Deep Agents CLI!

You: Create a 7-day meal plan for a vegetarian athlete

Agent: I'll create a comprehensive meal plan for you. Let me:
1. Research vegetarian athlete nutrition needs
2. Design balanced daily menus
3. Save the plan to a file

[Agent uses tools...]

Agent: I've created your meal plan! You can find it at:
workspace/vegetarian_athlete_meal_plan.md

You: /exit
```

In [22]:
# Check if CLI is installed
import subprocess

try:
    result = subprocess.run(["deepagents", "--version"], capture_output=True, text=True)
    print(f"deepagents-cli version: {result.stdout.strip()}")
except FileNotFoundError:
    print("deepagents-cli not installed. Install with:")
    print("  uv pip install deepagents-cli")
    print("  # or")
    print("  pip install deepagents-cli")

deepagents-cli version: deepagents-cli 0.0.20


### Try It Yourself!

After installing the CLI, try these commands in your terminal:

```bash
# Basic interactive session
deepagents

# With a specific working directory
deepagents --workdir ./workspace

# See all options
deepagents --help
```

Sample prompts to try:
1. "Create a weekly workout plan and save it to a file"
2. "Research the health benefits of meditation and summarize in a report"
3. "Analyze my current diet and suggest improvements" (then provide details)

## Task 10: Building a Complete Deep Agent System

Now let's bring together all four elements to build a comprehensive "Wellness Coach" system:

1. **Planning**: Track multi-week wellness programs
2. **Context Management**: Store session notes and progress
3. **Subagent Spawning**: Delegate to specialists (exercise, nutrition, mindfulness)
4. **Long-term Memory**: Remember user preferences and history

In [30]:
# Define specialized wellness subagents
# Subagents inherit the backend from the parent, so they use the same workspace
exercise_specialist = {
    "name": "exercise-specialist",
    "description": "Expert in exercise science, workout programming, and physical fitness. Use for exercise-related questions and plan creation.",
    "system_prompt": """You are an exercise specialist with expertise in:
- Workout programming for different fitness levels
- Exercise form and safety
- Progressive overload principles
- Recovery and injury prevention

Always consider the user's fitness level and any physical limitations.
Provide clear, actionable exercise instructions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

nutrition_specialist = {
    "name": "nutrition-specialist",
    "description": "Expert in nutrition science, meal planning, and dietary optimization. Use for food-related questions and meal plans.",
    "system_prompt": """You are a nutrition specialist with expertise in:
- Macro and micronutrient balance
- Meal planning and preparation
- Dietary restrictions and alternatives
- Nutrition timing for performance

Always respect dietary restrictions and preferences.
Focus on practical, achievable meal suggestions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

mindfulness_specialist = {
    "name": "mindfulness-specialist",
    "description": "Expert in stress management, sleep optimization, and mental wellness. Use for stress, sleep, and mental health questions.",
    "system_prompt": """You are a mindfulness and mental wellness specialist with expertise in:
- Stress reduction techniques
- Sleep hygiene and optimization
- Meditation and breathing exercises
- Work-life balance strategies

Be supportive and non-judgmental.
Provide practical techniques that can be implemented immediately.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

print("Specialist subagents defined!")

Specialist subagents defined!


In [31]:
# Create the Wellness Coach coordinator
wellness_coach = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        # Planning
        write_todos,
        update_todo,
        list_todos,
        # Long-term Memory
        get_user_profile,
        save_user_preference,
        # Skills
        load_skill,
    ],
    backend=filesystem_backend,  # All file ops go to workspace
    subagents=[exercise_specialist, nutrition_specialist, mindfulness_specialist],
    system_prompt="""You are a Personal Wellness Coach that coordinates comprehensive wellness programs.

## Your Role
- Understand each user's unique goals, constraints, and preferences
- Create personalized, multi-week wellness programs
- Coordinate between exercise, nutrition, and mindfulness specialists
- Track progress and adapt recommendations

## Workflow
1. **Initial Assessment**: Get user profile and understand their situation
2. **Planning**: Create a todo list for the program components
3. **Delegation**: Use specialists for domain-specific content:
   - exercise-specialist: Workout plans and fitness guidance
   - nutrition-specialist: Meal plans and dietary advice
   - mindfulness-specialist: Stress and sleep optimization
4. **Integration**: Combine specialist outputs into a cohesive program
5. **Documentation**: Save all plans and recommendations to files

## Important
- Always check user profile first for context
- Respect any medical conditions or dietary restrictions
- Provide clear, actionable recommendations
- Save progress to files so users can reference later"""
)

print("Wellness Coach created with all 4 Deep Agent elements!")

Wellness Coach created with all 4 Deep Agent elements!


In [35]:
# Test the complete system
TODO_STORE.clear()

result = wellness_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! My user_id is user_alex. I'd like you to create a 2-week wellness program for me.

I want to focus on:
1. Building a consistent exercise routine (I can exercise 3x per week for 30 mins)
2. Improving my diet (remember I'm vegetarian)
3. Better managing my work stress and improving my sleep

Please create comprehensive plans for each area and save them as separate files I can reference."""
    }]
})

print("Wellness Coach response:")
print(result["messages"][-1].content)

Wellness Coach response:
Perfect! Alex, your comprehensive 2-week wellness program is complete and organized! 🎉

## What I've Created For You:

### **Main Program File:**
- **Alex_Complete_2Week_Wellness_Program.md** - Your master guide with daily integration schedules, weekly progression, and success tracking

### **Individual Reference Files:**
- **Alex_Exercise_Plan.md** - Complete 3x/week workout routines with progressions
- **Alex_Nutrition_Plan.md** - 14 days of vegetarian meals with shopping lists and prep tips  
- **Alex_Mindfulness_Plan.md** - Daily stress management and sleep optimization techniques

### **Additional Resources:**
- Shopping lists for both weeks
- Meal prep guidelines
- Progress tracking suggestions

## Key Program Features:

✅ **Tailored to Your Preferences**: Morning exercise, vegetarian diet, detailed guidance
✅ **Addresses Your Goals**: Consistent routine, energy improvement, stress/sleep management  
✅ **Considers Your Conditions**: Anxiety-friendly techn

In [36]:
# Review what was created
print("=" * 60)
print("FINAL TODO STATUS")
print("=" * 60)
print(list_todos.invoke({}))

print("\n" + "=" * 60)
print("GENERATED FILES")
print("=" * 60)
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

FINAL TODO STATUS
✅ [todo_1] Assess user profile and preferences (completed)
✅ [todo_3] Create exercise plan (completed)
✅ [todo_5] Create nutrition plan (completed)
✅ [todo_7] Create stress & sleep optimization plan (completed)
✅ [todo_9] Integrate all plans into cohesive program (completed)
✅ [todo_11] Save individual plans to files (completed)
✅ [todo_13] Save complete program overview (completed)

GENERATED FILES
  [FILE] .gitkeep (0 bytes)
  [FILE] Alex_2_Week_Exercise_Program.txt (4330 bytes)
  [FILE] Alex_Complete_2Week_Wellness_Program.md (4757 bytes)
  [FILE] Alex_Exercise_Plan.md (3864 bytes)
  [FILE] Alex_Mindfulness_Plan.md (6396 bytes)
  [FILE] Alex_Nutrition_Plan.md (5482 bytes)
  [FILE] complete_14_day_vegetarian_meal_plan.txt (5045 bytes)
  [FILE] comprehensive_morning_routine_guide.md (25117 bytes)
  [FILE] meal_plan_nutritional_highlights.txt (468 bytes)
  [FILE] meal_plan_week_1.txt (1765 bytes)
  [FILE] meal_plan_week_2.txt (1598 bytes)
  [FILE] meal_prep_tips.txt (

In [37]:
# Read one of the generated files
files = list(WORKSPACE.glob("*.md"))
if files:
    print(f"\nContents of {files[0].name}:")
    print("=" * 60)
    print(files[0].read_text()[:2000] + "..." if len(files[0].read_text()) > 2000 else files[0].read_text())


Contents of morning_routine_guide.md:
# The Ultimate Morning Routine Guide: Energize Your Day for Peak Performance

## Table of Contents
- [Introduction: Why Your Morning Sets the Tone](#introduction-why-your-morning-sets-the-tone)
- [The Science-Backed Foundation](#the-science-backed-foundation)
- [Exercise: Wake Up Your Body](#exercise-wake-up-your-body)
- [Nutrition: Fuel Your Success](#nutrition-fuel-your-success)
- [Mindset Practices: Train Your Mental Muscle](#mindset-practices-train-your-mental-muscle)
- [Implementation Strategies: Building Lasting Habits](#implementation-strategies-building-lasting-habits)
- [Common Mistakes to Avoid](#common-mistakes-to-avoid)
- [Customization for Your Lifestyle](#customization-for-your-lifestyle)
- [Sample Morning Routine Templates](#sample-morning-routine-templates)
- [Your Next Steps](#your-next-steps)

---

## Introduction: Why Your Morning Sets the Tone

Picture this: You wake up feeling refreshed, energized, and ready to tackle whatever

---
## ❓ Question #3:

What are the key considerations when designing **subagent configurations**?

Consider:
- When should subagents share tools vs have distinct tools?
- How do you decide which model to use for each subagent?
- What's the right granularity for subagent specialization?

##### Answer:
When designing subagents it's important to be clear in their description and relevant tools so that the supervisor agent can pick the right subagent for a specific subtask.

Each subagent should have a focused tool set based on their specialisation, hence some tools can be distinct for some subagents. If there are actions that should be shared across subagents, e.g. two subagents writing files with a different focus, then some tools can be shared.

Models should be chosen accounting for their capability vs task as well as their cost. For example, if a subagent needs to do a task that requires complex maths skills, then we probably need a strong model that costs more. But if we need to do some simple task like a summary of straighforward content, then probably a smaller cheaper model would be most efficient.

Subagents should be designed in a way that each one has a clear distinct scope and focuses on its own task. However, as with to-do lists, one doesn't want to create many subagents that do some minute tasks - that would be a waste of resources. Similarly, defining too broad scope for a subagent is not a good idea either as it's not really specialised then.

## ❓ Question #4:

For a **production wellness application** using Deep Agents, what would you need to add?

Consider:
- Safety guardrails for health advice
- Persistent storage (not in-memory)
- Multi-user support and isolation
- Monitoring and observability
- Cost management with subagents

##### Answer:
Safety guardrails and data privacy would be the main considerations from the UX perspective and regulatory compliance. I would build guardrails into the master prompt of the supervisor agent to make sure that any content meets the specified criteria before it's presented back to the user.

I would use persistent storage to keep the user data in a secure way and would only inject user data where relevant. I would have separate entries for each user in the storage and make sure other users can't see other people's data.

I would use LangSmith to analyse logs and evaluate my workflows as well as models' decisions for optimisation (fixing errors, optimising workflows, costs, latency).

I would start with the smallest and cheapest model for my use case and consider more expensive ones only when the smaller ones underperform (after solid ICL attempts and thorough evaluation). I would use smaller and cheaper models for easier tasks (less complex subagents) and more expensive models for complex tasks and the supervisor.

---
## 🏗️ Activity #2: Build a Wellness Coach Agent

Build your own wellness coach that uses all 4 Deep Agent elements.

### Requirements:
1. **Planning**: Create todos for a 30-day wellness challenge
2. **Context Management**: Store daily check-in notes
3. **Subagents**: At least 2 specialized subagents
4. **Memory**: Remember user preferences across interactions

### Challenge:
Create a "30-Day Wellness Challenge" system that:
- Generates a personalized 30-day plan
- Tracks daily progress
- Adapts recommendations based on feedback
- Saves a weekly summary report

**I decided to build: Free Yourself From Back Pain Coach**

In [32]:
# Define subagent configurations

training_specialist = { 
    "name": "training-specialist",
    "description": "Expert in movement science, personal training, and injury prevention. Use for training-related questions and training plan creation.",
    "system_prompt": """You are a personal trainer specialist with expertise in:
- Personal training
- Exercise plans
- Back strength
- Core Strength
- Strength training
- Mobility training
- Movement science
- Injury prevention

Always consider the user's health history and any physical limitations.
Provide clear, actionable movement instructions.""",
    "tools": [],
    "model": "openai:gpt-4o-mini",
}

posture_specialist = {
    "name": "posture-specialist",
    "description": "Expert in posture correction, alignment, spinal health and rehabilitation. Use for posture-related and physiotherapy questions.",
    "system_prompt": """You are a posture specialist with expertise in:
- Posture correction
- Alignment
- Spinal health
- Postural awareness
- Rehabilitation

Always consider the user's posture, health history including any injuries, physical limitations and their lifestyle.
Provide clear, actionable posture instructions.""",
    "tools": [],
    "model": "openai:gpt-4o-mini",
}

pain_relief_specialist = {
    "name": "pain-relief-specialist",
    "description": "Expert in pain relief, massage therapy, and pain reduction. Use for pain-related questions and pain-relief treatment advice.",
    "system_prompt": """You are a pain relief specialist with expertise in:
- Pain relief
- Massage therapy (deep tissue, trigger point, sports massage, dry needling, lymphatic drainage, accupuncture, cupping, etc.)
- Pain management (including bodymind connection and pain science)

Always consider the user's pain history, pain severity, history of injuries, physical limitations and their lifestyle.
Provide clear, actionable pain relief instructions.""",
    "tools": [],
    "model": "openai:gpt-4o-mini",
}

In [33]:
# Build the coordinator agent
back_pain_coach = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        # Planning
        write_todos,
        update_todo,
        list_todos,
        # Long-term Memory
        get_user_profile,
        save_user_preference,
        # Skills
        load_skill,
    ],
    backend=filesystem_backend,
    subagents=[training_specialist, posture_specialist, pain_relief_specialist],
    system_prompt="""You are a Personal Back Pain Coach that coordinates a comprehensive 30 day program for people to free themselves from back pain.
        
    # Your Role:
    - You need to understand the user profile well to create a comprehensive 30 day plan that addresses user's needs in the three key areas: personal training, posture work and pain relief.
    - You will need to use the following subagents and coordinate their work to achieve this goal: training-specialist, posture-specialist, pain-relief-specialist.
    - You need to create a daily check-in with the user to track their progress and adapt the plan accordingly. Store daily check-in notes in a file.
    - You need to save a weekly summary report of the program progress in a file.


    # Workflow:
    1. **Initial Assessment**: Get user profile and understand their situation
    2. **Planning**: Create a todo list for the program components
    3. **Delegation**: Use specialists for domain-specific content:
        - training-specialist: Workout plans and training guidance
        - posture-specialist: Posture correction and alignment
        - pain-relief-specialist: Pain relief and massage therapy
    4. **Integration**: Combine specialist outputs into a cohesive program
    5. **Documentation**: Save all plans and recommendations to files

    # Important:
    - Always check user profile first for context
    - Respect any medical conditions, injuries, and physical limitations 
    - Provide clear, actionable recommendations
    - Save progress to files so users can reference later"""
)

In [34]:
# Add a user profile to long-term memory
# Ruth 
user_id = "ruth" 
profile_namespace = (user_id, "profile")

memory_store.put(profile_namespace, "name", {"value": "Ruth"})
memory_store.put(profile_namespace, "age", {"value": "30"})
memory_store.put(profile_namespace, "lifestyle", {
    "work": "remote at desk, sits most of the day",
    "activity": "sometimes yoga, no structured exercise routine",
})
memory_store.put(profile_namespace, "pain", {
    "duration": "about a year",
    "areas": "upper back and neck",
    "worse_with": "gardening",
})
memory_store.put(profile_namespace, "conditions", {
    "injuries": "none",
    "stress": "high",
    "sleep": "poor",
})
memory_store.put(profile_namespace, "goals", {"value": "get rid of pain and get relief"})

print(f"Stored profile for {user_id}")

Stored profile for ruth


In [35]:
# Test the back pain coach
TODO_STORE.clear()

user_id = "ruth"

back_coach_result = back_pain_coach.invoke({
    "messages": [{
        "role": "user",
        "content": f"""My name is Ruth, my user_id is {user_id}. I'm 30 years old. I work remotely at my desk and I sit most of my day. I've had pain for the last year, especially in my upper back and neck. I sometimes go to yoga classes but I have not had a structured exercise routine. I don't have any injuries but my back gets worse when I do gardening. I want to get rid of the pain and get some relief. I think high stress might be related. I don't sleep well to be honest."""
    }]
})
messages = back_coach_result["messages"]
for message in messages:
    print(message.content)

My name is Ruth, my user_id is ruth. I'm 30 years old. I work remotely at my desk and I sit most of my day. I've had pain for the last year, especially in my upper back and neck. I sometimes go to yoga classes but I have not had a structured exercise routine. I don't have any injuries but my back gets worse when I do gardening. I want to get rid of the pain and get some relief. I think high stress might be related. I don't sleep well to be honest.
[{'text': "Hello Ruth! I'm here to help you create a comprehensive 30-day program to address your back pain. Let me start by getting your current profile and then we'll work together to develop a personalized plan.", 'type': 'text'}, {'id': 'toolu_01WSJF2yuuzWTaQkoAohJiq3', 'input': {'user_id': 'ruth'}, 'name': 'get_user_profile', 'type': 'tool_use'}]
Profile for ruth:
  name: {'value': 'Ruth'}
  age: {'value': '30'}
  lifestyle: {'work': 'remote at desk, sits most of the day', 'activity': 'sometimes yoga, no structured exercise routine'}
  p

In [36]:
# Simulate a daily check-in 

check_in_result = back_pain_coach.invoke({
    "messages": [{
        "role": "user",
        "content": f"""Hi it's Ruth again, my user_id is {user_id}. It's the 3 day anf I wanna check-in: I've been doing the exercises and they relieve pain. But I need some other pain relief."""
    }]
})

for m in reversed(check_in_result["messages"]):
    if getattr(m, "content", None) and isinstance(m.content, str) and m.content.strip():
        print(m.content)
        break

Perfect! I've documented your day 3 progress. It's really encouraging that the exercises are working - that tells me we're on the right track with your program.

**Quick question for tomorrow:** Which of these new pain relief techniques feels most doable for you to try first? I'd suggest starting with just 1-2 so you don't feel overwhelmed, and we can build from there.

Keep up the great work with your exercises, and remember - the combination of movement + these additional pain relief strategies should give you even better results! 

How are you feeling about adding these new techniques to your routine?


---
## Summary

In this session, we explored **Deep Agents** and their four key elements:

| Element | Purpose | Implementation |
|---------|---------|----------------|
| **Planning** | Track complex tasks | `write_todos`, `update_todo`, `list_todos` |
| **Context Management** | Handle large contexts | File system tools, automatic offloading |
| **Subagent Spawning** | Delegate to specialists | `task` tool with custom configs |
| **Long-term Memory** | Remember across sessions | LangGraph Store integration |

### Key Takeaways:

1. **Deep Agents handle complexity** - Unlike simple tool loops, they can manage long-horizon, multi-step tasks
2. **Planning is context engineering** - Todo lists and files aren't just organization—they're extended memory
3. **Subagents prevent context bloat** - Delegation keeps the main agent focused and efficient
4. **Skills enable progressive disclosure** - Load capabilities on-demand instead of upfront
5. **The CLI makes interaction natural** - Interactive sessions with conversation resume

### Deep Agents vs Traditional Agents

| Aspect | Traditional Agent | Deep Agent |
|--------|-------------------|------------|
| Task complexity | Simple, single-step | Complex, multi-step |
| Context management | All in conversation | Files + summaries |
| Delegation | None | Subagent spawning |
| Memory | Within thread | Across sessions |
| Planning | Implicit | Explicit (todos) |

### When to Use Deep Agents

**Use Deep Agents when:**
- Tasks require multiple steps or phases
- Context would overflow in a simple loop
- Specialization would improve quality
- Users need to resume sessions
- Long-term memory is valuable

**Use Simple Agents when:**
- Tasks are straightforward Q&A
- Single tool call suffices
- Context fits easily
- No need for persistence

### Further Reading

- [Deep Agents Documentation](https://docs.langchain.com/oss/python/deepagents/overview)
- [Deep Agents GitHub](https://github.com/langchain-ai/deepagents)
- [Context Management Blog Post](https://www.blog.langchain.com/context-management-for-deepagents/)
- [Building Multi-Agent Applications](https://www.blog.langchain.com/building-multi-agent-applications-with-deep-agents/)
- [LangGraph Memory Concepts](https://langchain-ai.github.io/langgraph/concepts/memory/)